## 練習時間
參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [1]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### 之前做過的處理

In [2]:
# 設定 data_path
dir_data = './data/'
f_app_train = os.path.join(dir_data, 'application_train.csv')
f_app_test = os.path.join(dir_data, 'application_test.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
# also apply to testing dataset
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# absolute the value of DAYS_BIRTH
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])

In [3]:
app_train.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,DAYS_EMPLOYED_ANOM
0,100002,1,0,0,1,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,1,0,1,0,False
1,100003,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,1,0,0,0,0,0,0,1,0,False
2,100004,0,1,1,1,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0,0,0,False
3,100006,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,0,0,0,0,0,False
4,100007,0,0,0,1,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0,0,0,False
5,100008,0,0,0,1,0,99000.0,490495.5,27517.5,454500.0,...,0,0,0,0,0,0,0,0,0,False
6,100009,0,0,1,1,1,171000.0,1560726.0,41301.0,1395000.0,...,0,0,0,0,0,0,0,0,0,False
7,100010,0,0,1,1,0,360000.0,1530000.0,42075.0,1530000.0,...,0,0,0,0,0,0,0,0,0,False
8,100011,0,0,0,1,0,112500.0,1019610.0,33826.5,913500.0,...,0,0,0,0,0,0,0,0,0,True
9,100012,0,1,0,1,0,135000.0,405000.0,20250.0,405000.0,...,0,0,0,0,0,0,0,0,0,False


In [22]:
app_train["equal_AMT_GOODS_PRICE"] = pd.cut(app_train["AMT_GOODS_PRICE"], 10)
app_train["equal_AMT_GOODS_PRICE"].value_counts()

(36490.5, 441450.0]       131712
(441450.0, 842400.0]      115261
(842400.0, 1243350.0]      43658
(1243350.0, 1644300.0]     12429
(1644300.0, 2045250.0]      3175
(2045250.0, 2446200.0]       933
(2446200.0, 2847150.0]        30
(2847150.0, 3248100.0]        17
(3649050.0, 4050000.0]        10
(3248100.0, 3649050.0]         8
Name: equal_AMT_GOODS_PRICE, dtype: int64

In [23]:
app_train['equal_freq_AMT_GOODS_PRICE'] = pd.qcut(app_train['AMT_GOODS_PRICE'], 10)
app_train['equal_freq_AMT_GOODS_PRICE'].value_counts()

(522000.0, 675000.0]      42417
(40499.999, 180000.0]     39484
(373500.0, 450000.0]      35052
(180000.0, 225000.0]      31970
(814500.0, 1093500.0]     30745
(1093500.0, 4050000.0]    30685
(225000.0, 270000.0]      27990
(450000.0, 522000.0]      26475
(270000.0, 373500.0]      23483
(675000.0, 814500.0]      18932
Name: equal_freq_AMT_GOODS_PRICE, dtype: int64

In [34]:
app_train["customized_AMT_GOODS_PRICE"] = pd.cut(app_train["AMT_GOODS_PRICE"], [1000, 5000, 10000, 50000, 100000, 500000, 1000000, 5000000, 10000000])
app_train[['equal_AMT_GOODS_PRICE', 'equal_freq_AMT_GOODS_PRICE', 'customized_AMT_GOODS_PRICE']].sort_values('equal_AMT_GOODS_PRICE', ascending=False).head(10)

,equal_AMT_GOODS_PRICE,equal_freq_AMT_GOODS_PRICE,customized_AMT_GOODS_PRICE
14852,"(3649050.0, 4050000.0]","(1093500.0, 4050000.0]","(1000000, 5000000]"
288455,"(3649050.0, 4050000.0]","(1093500.0, 4050000.0]","(1000000, 5000000]"
120757,"(3649050.0, 4050000.0]","(1093500.0, 4050000.0]","(1000000, 5000000]"
14643,"(3649050.0, 4050000.0]","(1093500.0, 4050000.0]","(1000000, 5000000]"
110459,"(3649050.0, 4050000.0]","(1093500.0, 4050000.0]","(1000000, 5000000]"
287732,"(3649050.0, 4050000.0]","(1093500.0, 4050000.0]","(1000000, 5000000]"
17948,"(3649050.0, 4050000.0]","(1093500.0, 4050000.0]","(1000000, 5000000]"
119681,"(3649050.0, 4050000.0]","(1093500.0, 4050000.0]","(1000000, 5000000]"
133766,"(3649050.0, 4050000.0]","(1093500.0, 4050000.0]","(1000000, 5000000]"
95571,"(3649050.0, 4050000.0]","(1093500.0, 4050000.0]","(1000000, 5000000]"
